# Melbourne Housing Prices

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import catboost
import shap
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

## Import Data

In [ ]:
df = pd.read_csv( './Data/MELBOURNE_HOUSE_PRICES_LESS.csv' )

## Explore Data

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
col = 'Price'

plt.hist( df[ col ], bins = 50 )
plt.title( 'Histogram of ' + col + ' Values' )
plt.show()

## Features to Use

In [ ]:
features = ['Rooms','Distance']
#features = ['Suburb','Method','CouncilArea','Rooms','Distance']

target_cols = ['Price']

In [ ]:
df_sifted = df[ features + target_cols ]

In [ ]:
df_sifted

## Clean Data

In [ ]:
df_sifted.dropna( inplace = True )  

In [ ]:
df_sifted

## Separate DF

In [ ]:
df_features = df_sifted[ features   ]
df_target   = df_sifted[ target_cols[0] ]

In [ ]:
df_features.head()

In [ ]:
df_target.head()

## Split into Train / Test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( df_features, df_target, test_size = .25 )

In [ ]:
X_train.head()

In [ ]:
y_train.head()

## Select a Model

In [ ]:
model = catboost.CatBoostRegressor()

In [ ]:
model.fit( X_train, y_train, verbose = False )

In [ ]:
df_train_results = pd.DataFrame( {'Estimated Home Value': model.predict( X_train ) }, index = X_train.index )
df_test_results = pd.DataFrame( {'Estimated Home Value': model.predict( X_test ) }, index = X_test.index )


In [ ]:
df_test = X_test.join( y_test )
df_test = df_test.join( df_test_results )

In [ ]:
df_test

In [ ]:
df_train = X_train.join( y_train )
df_train = df_train.join( df_train_results )

In [ ]:
df_train

In [ ]:
mean_absolute_error( [10,9,8], [3,4,5] )

## Check the Model Fit

In [ ]:
print ('TRAIN: ' + str( mean_absolute_error( df_train['Price'], df_train['Estimated Home Value'] ) ))
print ('TEST: ' + str( mean_absolute_error( df_test['Price'], df_test['Estimated Home Value'] ) ))

## Shap

In [ ]:
explainer = shap.TreeExplainer( model )
shap_values = explainer.shap_values( df_features )
shap.summary_plot(shap_values, df_features, show = False )
plt.gcf().set_size_inches( 12.0, 12.0)
plt.show()

